In [3]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 3.8 MB/s 


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 29.8 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter
import torchmetrics
import matplotlib.pyplot as plt

import pandas as pd
import transformers
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
# загрузка модели
model_bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
print(model_bert)
print("Parameters full train:", sum([param.nelement() for param in model_bert.parameters()]))

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [7]:
#пример классификации сентимента текста
sentiment = pipeline("text-classification", model='SkolkovoInstitute/russian_toxicity_classifier')
sentiment("Этот ресторан отличный")

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'neutral', 'score': 0.998418927192688}]

In [8]:
#пример работы токенизации
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

example_text = 'Пример текста для токенизации'

bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 17863, 10316,   949, 15427,   831,  2703,   102,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [9]:
tokenizer.ids_to_tokens[101], tokenizer.ids_to_tokens[17863], tokenizer.ids_to_tokens[10316]

('[CLS]', 'Пример', 'текста')

In [10]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD] [PAD]


In [11]:
df_train = pd.read_csv("/content/drive/MyDrive/HW_6/data/train.csv")
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [12]:
df_val = pd.read_csv("/content/drive/MyDrive/HW_6/data/val.csv")
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [13]:
sentiment = pipeline("text-classification", model='SkolkovoInstitute/russian_toxicity_classifier')

idx = 0
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
print('label by model is', sentiment(df_train.iloc[idx]['text'])[0]['label'], 'with score', sentiment(df_train.iloc[idx]['text'])[0]['score'])

@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.
label is 0
label by model is neutral with score 0.9937475919723511


In [14]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [15]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=0)

In [16]:
for txt, lbl in train_loader:
    print(txt.keys()) #словарь с ключами'input_ids', 'token_type_ids', 'attention_mask'
    print(txt['input_ids'].shape) #тензор размера (B,1,max_len) из id токенов
    print(txt['attention_mask'].shape) #тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внимание
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])
torch.Size([64, 1, 10])


In [17]:
#метрика модели на валидационном датасете
model_bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

valid_f1 = torchmetrics.F1Score()

for val_input, val_label in valid_loader:
    val_label = val_label
    mask = val_input['attention_mask'] 
    input_id = val_input['input_ids'].squeeze(1)
    
    output = model_bert(input_id, mask)[0]
    
    valid_f1(output, val_label)
    
print(f'Val f1_score: {valid_f1.compute().item():.3f}')

Val f1_score: 0.486


In [18]:
#инициализация модели классификации сентимента текста
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.classifier.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [19]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = Adam(model.classifier.parameters(), lr=0.001) 

In [20]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах

train_f1 = torchmetrics.F1Score()
valid_f1 = torchmetrics.F1Score()

epochs = 1

for epoch in range(epochs):
    model.train()
    total_loss_train = 0.0
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask']
        input_id = train_input['input_ids'].squeeze(1)
        train_label = train_label

        output = model(input_id, mask)[0]
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        train_f1(output, train_label)
        
        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val = 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label
        mask = val_input['attention_mask']
        input_id = val_input['input_ids'].squeeze(1)

        output = model(input_id, mask)[0]

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        valid_f1(output, val_label)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val f1: {valid_f1.compute().item(): .3f}')
    train_f1.reset()
    valid_f1.reset()

100%|██████████| 2836/2836 [4:50:23<00:00,  6.14s/it]


Epochs: 1 | Train Loss:  0.011         | Train f1:  0.577         | Val Loss:  0.010         | Val f1:  0.576


In [21]:
#метрика дообученной модели на валидационном датасете
valid_f1 = torchmetrics.F1Score()
model.eval()

for val_input, val_label in valid_loader:
    val_label = val_label
    mask = val_input['attention_mask'] 
    input_id = val_input['input_ids'].squeeze(1)
    
    output = model(input_id, mask)[0]
    
    valid_f1(output, val_label)
    
print(f'Val f1_score: {valid_f1.compute().item():.3f}')

Val f1_score: 0.576


In [22]:
#класс модели BertClassifier
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.bert(input_ids=x, attention_mask=mask,return_dict=False)[0]  #(B, 2)
        final_layer = self.sigm(pooled_output)
        return final_layer

In [23]:
#инициализация модели
model_sigm = BertClassifier()
print(model_sigm)
print("Parameters full train:", sum([param.nelement() for param in model_sigm.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model_sigm.bert.classifier.parameters()]))

BertClassifier(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tr

In [24]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
#дообучаем последний слой классификации
optimizer = Adam(model_sigm.bert.classifier.parameters(), lr=0.001)

In [25]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах

train_f1 = torchmetrics.F1Score()
valid_f1 = torchmetrics.F1Score()

epochs = 1

for epoch in range(epochs):
    model_sigm.train()
    total_loss_train = 0.0
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask']
        input_id = train_input['input_ids'].squeeze(1)
        train_label = train_label

        output = model_sigm(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        train_f1(output, train_label)
        
        model_sigm.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model_sigm.eval()
    total_loss_val = 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label
        mask = val_input['attention_mask']
        input_id = val_input['input_ids'].squeeze(1)

        output = model_sigm(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        valid_f1(output, val_label)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val f1: {valid_f1.compute().item(): .3f}')
    train_f1.reset()
    valid_f1.reset()

100%|██████████| 2836/2836 [4:51:53<00:00,  6.18s/it]


Epochs: 1 | Train Loss:  0.011         | Train f1:  0.585         | Val Loss:  0.010         | Val f1:  0.583


In [26]:
#метрика дообученной модели на валидационном датасете
valid_f1 = torchmetrics.F1Score()
model_sigm.eval()

for val_input, val_label in valid_loader:
    val_label = val_label
    mask = val_input['attention_mask'] 
    input_id = val_input['input_ids'].squeeze(1)
    
    output = model_sigm(input_id, mask)
    
    valid_f1(output, val_label)
    
print(f'Val f1_score: {valid_f1.compute().item():.3f}')

Val f1_score: 0.583
